In [1]:
import requests
from bs4 import BeautifulSoup

In [3]:
# https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex=31
url ='https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'

def pageindex():
    pageIndex = []
    for i in range(1,30): #29P까지
        pageIndex.append (i)
    print (pageIndex)
    return pageIndex

In [2]:
import PyPDF2
import re

In [ ]:
def bok():
    
    import re
    
    pageIndex= pageindex()
    bs_list = []
    
    for i in range(len(pageIndex)):

        url ='https://www.bok.or.kr/portal/bbs/B0000245/list.do?'

        params = ({
             'menuNo' : '200761',
             'pageIndex': pageIndex[i]
         })

        resp = requests.get(url,params)
        soup = BeautifulSoup(resp.content)

        find_soup = soup.find('div',class_ = 'bdLine type2')

        all_li_List = find_soup.find_all('li')

    #=================#print('all_li',len(all_li_List))========================================    
        find_li_List = []

        for j in range(len(all_li_List)):
            try :
                temp = all_li_List[j].find('div',class_ = "row")

                if temp:
                    find_li_List.append(temp) 

            except Exception:
                pass
        print(len(find_li_List))# 한페이지에 가져오는 리스트수
    #=============================================================       

        for j in range(len(find_li_List)):

            try:
                title_dates  = find_li_List[j].find('span',class_ = "titlesub").text
                upload_dates = find_li_List[j].find('span',class_ = "date").text
                pdf_link= find_li_List[j].find('div',class_ = "fileGoupBox").find('a')['href']
                pdf_title= find_li_List[j].find('div',class_ = "fileGoupBox").find('a').text


                if pdf_title.find('pdf')== -1:
                    pdf_link = pdf_link[:len(pdf_link)-1]+'2'

                pdf_link = 'https://www.bok.or.kr'+pdf_link
                file_path = 'E:\\BOK\\{0}_{1}.pdf'.format(title_dates, upload_dates)
                file_result_path = 'E:\\BOK\\result{0}_{1}.txt'.format(title_dates, upload_dates)
            except Exception:
                pass
    #===============pdf download=======================
    #         downloads_pdf(pdf_link,file_path) #피디에프 다운로드 함수화

    #===============pdf to text=======================
    #         text = convert_pdf_to_txt(file_path)
    #         print (text)
    #=================================================
            write_file_text(file_result_path)
    #=================파일로저장======================   
            bs_list.append({
                'Title Date':title_dates,
                'Upload Date':upload_dates,
                'pdf_link': pdf_link,
                'pdf_title':pdf_title
            })


    
    return bs_list
    


In [1]:
def write_file_text(path): #저장될 파일 path
    f= open(path,'wt',encoding='UTF8') 
    f.write(text)
    f.close
    

In [ ]:
bok_list = bok():
print(bok_list)
len(bok_list)

In [ ]:
from pprint import pprint

pprint(bok_list)

In [ ]:
def downloads_pdf(pdf_link,file_path):
    file_res = requests.get(pdf_link)
    
    with open(file_path,'wb') as f: 
        f.write(file_res.content)


In [ ]:
# len(bs_list)

In [4]:
# !pip install pdfminer

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
#   pdf 리소스 매니저 객체 생성
    rsrcmgr = PDFResourceManager()
    
    retstr = StringIO()     
    #문자열 데이터를 파일처럼 처리하는 stringio -> pdf 파일 내용이 여기 담김
    
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, 
                                  pagenos, 
                                  maxpages=maxpages, 
                                  password=password,
                                  caching=caching, 
                                  check_extractable=True):
        
        interpreter.process_page(page)

    text = retstr.getvalue() #text 에 결과 담김 

    fp.close()
    device.close()
    retstr.close()
    return text